In [1]:
!rm -rf MQRNN/
!git clone https://github.com/tianchen101/MQRNN.git

In [12]:
# import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from MQRNN import Encoder, Decoder
# from MQRNN.MQRNN import MQRNN
# from MQRNN.data import MQRNN_dataset, read_df

In [ ]:
config = {
    'horizon_size': 40,
    'hidden_size': 50,
    'quantiles': [0.8], 
    'columns': [1],
    'dropout': 0.3,
    'layer_size': 2,
    'by_direction': False,
    'lr': 1e-3,
    'batch_size': 1,
    'num_epochs': 500,
    'context_size': 10,
}

# Thiết lập device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [6]:
train = pd.read_csv('./data/rossmann-store-sales/train.csv', low_memory=False)
test = pd.read_csv('./data/rossmann-store-sales/test.csv', low_memory=False)
store = pd.read_csv('./data/rossmann-store-sales/store.csv', low_memory=False)
print("Successfully loaded datasets.")

Successfully loaded datasets.


In [9]:
test.fillna(1,inplace=True)

store.CompetitionDistance = store.CompetitionDistance.fillna(store.CompetitionDistance.median())
store.fillna(0,inplace=True)
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0


In [10]:
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

In [11]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
